In [23]:
import sys
sys.path.append('../_amigocloud')
from amigocloud import AmigoCloud

import pandas as pd
from datetime import datetime, timedelta
import collections

from docxtpl import DocxTemplate
from docx.shared import Mm
import docxtpl
import requests
import os

In [24]:
def get_unidad_local():
    try:
        # ruta del archivo que contiene la información necesaria
        with open('../../unidad_local.txt', 'r') as file:
            unidad = file.readline().strip()
            return unidad
    except FileNotFoundError:
        print("El archivo 'unidad_local.txt' no se encuentra en la ruta especificada.")
    return None

def get_ruta_local():
    try:
        with open('../ruta_local.txt', 'r') as file:
            ruta = file.readline().strip()
            return ruta
    except FileNotFoundError:
        print("El archivo 'ruta_local.txt' no se encuentra en la ruta especificada.")
    return None

In [25]:
# RUTA ONEDRIVE
unidad = get_unidad_local()
ruta = get_ruta_local()
ruta_completa = os.path.join(unidad, ruta)
ruta_completa

'C:\\Documents\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON'

In [26]:
# leer token
acceso = open(ruta_completa + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [27]:
def crear_lista_imagenes(id_proyecto, source_amigo_id, doc):
    print('buscando fotos')
    query_fotos = {'query': 'select * from gallery_67001 where source_amigo_id=\'{amigo_id}\''.format(amigo_id=source_amigo_id)}
    print(query_fotos)
    project = 'https://app.amigocloud.com/api/v1/projects/' + id_proyecto + '/sql'
    select_fotos = amigocloud.get(project, query_fotos)
    fotos = select_fotos['data']
    lista_fotos_inline = []
    print('descargando fotos')
    for foto in fotos:
        print('descargando una foto')
        response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['s3_filename'])
        file = open(ruta_completa + '/fotos/' + foto['s3_filename'], "wb")
        file.write(response.content)
        file.close()
        lista_fotos_inline.append({'foto': docxtpl.InlineImage(doc, image_descriptor=ruta_completa +'/fotos/' + foto['s3_filename'], width=Mm(60))})
    return lista_fotos_inline

In [28]:
def generar_reporte(cod):
    id = cod
    query = {'query': 'select mu.*, (select count(*) from dataset_365891 da where mu.amigo_id = da.muestra_ref_datos) from dataset_365887 mu where id={idd}'.format(idd=id)}
    select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/35567/sql', query)
    data = select['data'][0]
    data['fecha_registro'] = datetime.strptime(data['fecha_registro'].split(' ')[0], '%Y-%m-%d').strftime('%d/%m/%Y')
    print(data)
    insp = {
        'fecha':data['fecha_registro'],
        'canhero':data['canhero'],
        'propiedad':data['propiedad'].split('--')[0],
        'lote':data['lote'],
        'area':data['area'],
        'variedad':data['variedad'],
        'edad':data['edad'],
        'ciclo':data['ciclo'],
        'num_mus':data['count']*20,
        'id':data['id'],
        'amigo_id':data['amigo_id']
    }
    insp = collections.namedtuple("insp", insp.keys())(*insp.values())  

    query = {'query': 'select * from dataset_365891 where muestra_ref_datos=\'{amigo_id}\''.format(amigo_id=data['amigo_id'])}
    select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/35567/sql', query)
    datos = select['data']
    tabla_datos = pd.DataFrame(datos)[['id', 'total_yemas', 'perforadas', 'enraizamiento', 'brotacion', 'muerta']]

    total_yemas = tabla_datos['total_yemas'].sum()
    total_perforadas = tabla_datos['perforadas'].sum()
    total_enraizamiento = tabla_datos['enraizamiento'].sum()
    total_brotacion = tabla_datos['brotacion'].sum()
    total_muertas = tabla_datos['muerta'].sum()
    total_inviables = total_perforadas + total_enraizamiento + total_brotacion + total_muertas
    total_viables = total_yemas - total_inviables

    porcen_perforadas = (total_perforadas / total_yemas)*100
    porcen_enraizamiento = (total_enraizamiento / total_yemas)*100
    porcen_brotacion = (total_brotacion / total_yemas)*100
    porcen_muertas = (total_muertas / total_yemas)*100
    porcen_inviables = (total_inviables / total_yemas)*100
    porcen_viables = (total_viables / total_yemas)*100

    res = {
        'diat':total_perforadas,
        'en':total_enraizamiento,
        'bro':total_brotacion,
        'mur':total_muertas,
        'invi':total_inviables,
        'via':total_viables,
        'ana':total_yemas,
        'diatp':porcen_perforadas,
        'enp':porcen_enraizamiento,
        'brop':porcen_brotacion,
        'murp':porcen_muertas,
        'invip':porcen_inviables,
        'viap':porcen_viables
    }
    res = collections.namedtuple("res", res.keys())(*res.values())
    
    doc = DocxTemplate(ruta_completa + "/templates/tpl_rpt_calidad_semilla.docx")
    id_proyecto = '35567'
    source_amigo_id = insp.amigo_id
    inline_fotos = crear_lista_imagenes(id_proyecto, source_amigo_id, doc)
    context = {'insp':insp, 'res':res, 'fotos':inline_fotos }
    doc.render(context)
    ca = insp.canhero.split(' / ')
    file_name = ca[0] + '_CDSE_' + insp.fecha.replace('/', '-') + '_' + ca[1] + '_' + str(insp.id) + '_' + str(insp.lote)
    doc.save(ruta_completa + '/informes/_' + file_name + '.docx')

In [22]:
query = {'query': 'select id, canhero, propiedad, lote, variedad from dataset_365887 where procesado=false'}
select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/35567/sql', query)
data = select['data']
tabla = pd.DataFrame(data)
tabla

,id,canhero,propiedad,lote,variedad
0,188,3473 / ESCALERA FRANCO JORGE,None,None,UCG 90-20


In [38]:
fecha_insp = '2025-03-29'
query = {'query': 'select id, canhero, propiedad, lote, variedad from dataset_365887 where fecha_registro::date=\'{fecha}\''.format(fecha=fecha_insp)}
select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/35567/sql', query)
data = select['data']
tabla = pd.DataFrame(data)
tabla

,id,canhero,propiedad,lote,variedad
0,196,8624 / MORENO TORRICO JOSE ALBERTO,54 / SANTA MARIA--MORENO TORRICO,L2.1,CITTCA 0563
1,197,8624 / MORENO TORRICO JOSE ALBERTO,54 / SANTA MARIA--MORENO TORRICO,L8.1,CITTCA 85-22
2,198,8624 / MORENO TORRICO JOSE ALBERTO,54 / SANTA MARIA--MORENO TORRICO,L8.2,CITTCA 85-22
3,199,8624 / MORENO TORRICO JOSE ALBERTO,54 / SANTA MARIA--MORENO TORRICO,L2.1,CITTCA 85-22


In [39]:
ids = list(tabla['id'])
ids

[196, 197, 198, 199]

In [40]:
for i in ids:
    generar_reporte(i)
    print(i)

{'fecha_registro': '29/03/2025', 'id': 196, 'estado': 'APROBADO', 'ubicacion': '0101000020E610000044F77EDD839B4FC0A8AB434CDF4831C0', 'tecnico_responsable': 'JUAN PABLO', 'canhero': '8624 / MORENO TORRICO JOSE ALBERTO', 'lote': 'L2.1', 'observaciones': None, 'propiedad': '54 / SANTA MARIA--MORENO TORRICO', 'variedad': 'CITTCA 0563', 'ciclo': 'HOJA', 'area': 4.0, 'edad': 10, 'usuario': 'Juan Pablo Jaldin Duran', 'amigo_id': '0968C9A23E324F90BE445FCCC0B006EC', 'procesado': True, 'count': 1}
buscando fotos
{'query': "select * from gallery_67001 where source_amigo_id='0968C9A23E324F90BE445FCCC0B006EC'"}
descargando fotos
descargando una foto
196
{'fecha_registro': '29/03/2025', 'id': 197, 'estado': 'APROBADO', 'ubicacion': '0101000020E6100000EE95FF8C059C4FC004071222A54A31C0', 'tecnico_responsable': 'JUAN PABLO', 'canhero': '8624 / MORENO TORRICO JOSE ALBERTO', 'lote': 'L8.1', 'observaciones': None, 'propiedad': '54 / SANTA MARIA--MORENO TORRICO', 'variedad': 'CITTCA 85-22', 'ciclo': 'SOCA 1